## Simulation - Sophie Ensing

In [1]:
# Package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# You need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# Spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style

# Package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Tranport network analysis package
import transport_network_analysis.core as core
import transport_network_analysis.graph_module as graph_module
import transport_network_analysis.model as model

# Used for mathematical functions
import math             

# Used for making the graph to visualize our problem
import networkx as nx  

### Import data

In [2]:
# Import stations and lines from excel sheets
df1 = pd.read_excel("../../Data.xlsx", sheet_name = "Stations")
df2 = pd.read_excel("../../Data.xlsx", sheet_name = "Lijnen")

# Convert columns to list type
from ast import literal_eval
df2.loc[:,'Route1'] = df2.loc[:,'Route1'].apply(lambda x: literal_eval(x))
df2.loc[:,'Route2'] = df2.loc[:,'Route2'].apply(lambda x: literal_eval(x))
df2.loc[:,'Duration'] = df2.loc[:,'Duration'].apply(lambda x: literal_eval(x))

In [3]:
# Create dictionary for all transport lines
lines = {}
durations = []

for index, row in df2.iterrows():
    lines[row["Line"]] = (row["Route1"], row["Route2"], row["Duration"])
    
edges = []
for k, v in lines.items():
    for x in range(len(v[0])):
        try:
            edges.append((v[0][x], v[0][x + 1], k, v[2][x]))
            edges.append((v[0][x + 1], v[0][x], k, v[2][x]))
        except:
            break

### Create graph

In [4]:
# Create graph
FG = nx.DiGraph()

# Create all stations and nodes
Node = type('Station', (core.Identifiable, core.Log, core.Locatable, core.HasResource, core.Station), {})
nodes = []
stations = []

for index, row in df1.iterrows():
    stations.append({"env": [],
                     "name": row["HalteNaam"],
                     "geometry": shapely.geometry.Point(row["HalteLat"], row["HalteLon"]),
                     "capacity": row["Capacity"]}) 

for station in stations:
    node = Node(**station)
    nodes.append(node)

# Create positions and add nodes to graph with right position
positions = {}

for node in nodes:
    positions[node.name] = (node.geometry.y, node.geometry.x)
    FG.add_node(node.name, geometry = node.geometry, object_type = node)

# Create all edges
for edge in edges:
    FG.add_edge(edge[0], edge[1], line = edge[2], duration = edge[3])

In [5]:
# Plot the network
nx.draw(FG, positions, node_size=10)
plt.show()

### Create moving parts

In [6]:
def generate_passenger(origin, destination, environment):
    """ Generate a passenger """
    Passenger = type('Passenger', (core.Identifiable, core.Routeable, core.Log), {})
    
    # Name of the object
    name = "Passenger"
    
    # Geometry of the origin
    geometry = nx.get_node_attributes(environment.FG, "object_type")[origin].geometry
    
    # Travel direction
    route = nx.dijkstra_path(environment.FG, origin, destination, weight = 'duration')
    
    return Passenger(**{"env": environment, 
                        "name": name, 
                        "route": route})

In [7]:
def generate_metro(line, environment):
    """ Generate a metro """
    Metro = type('Metro', (core.Identifiable, core.Movable, core.HasContainer, core.HasResource, 
                           core.Routeable, core.Mover), {})

    # Name of the object
    name = "Metro"

    # Travel direction
#     route = nx.dijkstra_path(environment.FG, origin, destination, weight = 'duration')
    route = line    
    # Geometry of origin

    geometry = nx.get_node_attributes(environment.FG, "object_type")[route[0]].geometry
    
    # Maximum passengers
    capacity = 50
    
    return Metro(**{"env": environment, 
                    "name": name, 
                    "route": route, 
                    "capacity": capacity, 
                    "geometry": geometry})

### Spawn the metro trains

In [8]:
def vehicle(env, start, frequency, line):
    yield env.timeout(start * 60 * 60)
    while True:
        metro = generate_metro(line, env)
        env.metros.append(metro)
        env.process(metro.move())
        
        # One metro every X minutes
        yield env.timeout(frequency * 60)

In [9]:
def passengers(env):
    yield env.timeout(5.5 * 60 * 60)
    while True:
        origin, destination = random.sample(env.FG.nodes, 2)

        passenger = generate_passenger(origin, destination, env)
        FG.nodes[origin]["object_type"].units.append(passenger)

        passenger.log_entry("Waiting for metro start", env.now, 0, nx.get_node_attributes(env.FG, "object_type")[origin].geometry)
        
        env.passengers.append(passenger)

        # Random arrivals of passengers with mean interarrival time of 30 seconds
        yield env.timeout(random.random() * 60)

### Run simulation

In [10]:
# Simulation start time
simulation_start = datetime.datetime(2019, 1, 1)

# Make simulation environment
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.FG = FG

# For loggin
env.metros = []
env.passengers = []

# Start the simulation
env.process(vehicle(env, 5.5, 6, lines["metro52"][0]))
env.process(vehicle(env, 5.585, 6, lines["metro52"][1]))
env.process(vehicle(env, 6, 10, lines["tram24"][0]))
env.process(vehicle(env, 6, 10, lines["tram24"][1]))
env.process(passengers(env))
env.run(until = env.now + 1 * 24 * 60 * 60)

### Check results

In [11]:
env.metros[56].log

{'Message': ['Driving from Noord to Noorderpark start',
  'Driving from Noord to Noorderpark stop',
  'Unloading start',
  'Unloading stop',
  'Driving from Noorderpark to CentraalStation start',
  'Driving from Noorderpark to CentraalStation stop',
  'Unloading start',
  'Unloading stop',
  'Driving from CentraalStation to Rokin start',
  'Driving from CentraalStation to Rokin stop',
  'Unloading start',
  'Unloading stop',
  'Driving from Rokin to Vijzelgracht start',
  'Driving from Rokin to Vijzelgracht stop',
  'Unloading start',
  'Unloading stop',
  'Loading start',
  'Loading stop',
  'Driving from Vijzelgracht to DePijp start',
  'Driving from Vijzelgracht to DePijp stop',
  'Unloading start',
  'Unloading stop',
  'Driving from DePijp to Europaplein start',
  'Driving from DePijp to Europaplein stop',
  'Unloading start',
  'Unloading stop',
  'Driving from Europaplein to Zuid start',
  'Driving from Europaplein to Zuid stop',
  'Unloading start',
  'Unloading stop'],
 'Times

In [12]:
# What routes did people (i = person) take that took this metro
for i in env.metros[10].units:
    print(i)
    print(i.route)

['Rokin', 'Vijzelgracht', 'DePijp', 'RoelofHartplein', 'GerritvdVeenstraat', 'Beethovenstraat', 'Minervaplein', 'Olympiaplein']
['RoelofHartplein', 'GerritvdVeenstraat', 'Beethovenstraat', 'Minervaplein', 'Olympiaplein']
['Minervaplein', 'Olympiaplein', 'Olympiaweg', 'OlympischStadion', 'IJsbaanpad', 'Amstelveenseweg', 'VUMedischCentrum']
['Amstelveenseweg', 'VUMedischCentrum']


In [13]:
env.metros[10].route

['CentraalStation',
 'Dam',
 'Rokin',
 'Muntplein',
 'Vijzelgracht',
 'MarieHeinekenplein',
 'DePijp',
 'RoelofHartplein',
 'GerritvdVeenstraat',
 'Beethovenstraat',
 'Minervaplein',
 'Olympiaplein',
 'Olympiaweg',
 'OlympischStadion',
 'IJsbaanpad',
 'Amstelveenseweg',
 'VUMedischCentrum',
 'DeBoelelaanVU']

In [14]:
for i in env.metros[10].log["Geometry"]:
    print(i)

POINT (52.37771998 4.90104337)
POINT (52.37771998 4.90104337)
POINT (52.37771998 4.90104337)
POINT (52.37492885 4.89549359)
POINT (52.37492885 4.89549359)
POINT (52.37492885 4.89549359)
POINT (52.37492885 4.89549359)
POINT (52.37492885 4.89549359)
POINT (52.37492885 4.89549359)
POINT (52.36985548 4.89230478)
POINT (52.36985548 4.89230478)
POINT (52.36985548 4.89230478)
POINT (52.36985548 4.89230478)
POINT (52.36985548 4.89230478)
POINT (52.36985548 4.89230478)
POINT (52.36646202 4.89323826)
POINT (52.36646202 4.89323826)
POINT (52.36646202 4.89323826)
POINT (52.36646202 4.89323826)
POINT (52.36646202 4.89323826)
POINT (52.36646202 4.89323826)
POINT (52.36029657 4.89111952)
POINT (52.36029657 4.89111952)
POINT (52.36029657 4.89111952)
POINT (52.36029657 4.89111952)
POINT (52.36029657 4.89111952)
POINT (52.36029657 4.89111952)
POINT (52.35722131 4.89077224)
POINT (52.35722131 4.89077224)
POINT (52.35722131 4.89077224)
POINT (52.35722131 4.89077224)
POINT (52.35272728 4.89073444)
POINT (5

In [15]:
env.passengers[9].__dict__

{'env': <simpy.core.Environment at 0x10fce2198>,
 'log': {'Message': ['Waiting for metro start'],
  'Timestamp': [datetime.datetime(2019, 1, 1, 5, 35, 43, 915646)],
  'Value': [0],
  'Geometry': [<shapely.geometry.point.Point at 0x10a86e080>]},
 'route': ['Vijzelgracht', 'Muntplein'],
 'complete_path': None,
 'name': 'Passenger',
 'id': '00278718-6c19-11e9-a036-7831c1d08f6a'}

In [16]:
passengerdf = pd.DataFrame.from_dict(env.passengers[4].log)
passengerdf

,Message,Timestamp,Value,Geometry
0,Waiting for metro start,2019-01-01 05:32:36.781971,0,POINT (52.34886793 4.87720177)
1,Waiting for metro stop,2019-01-01 06:19:00.000000,0,POINT (52.34886793 4.87720177)
2,In metro start,2019-01-01 06:19:00.000000,0,POINT (52.34886793 4.87720177)
3,In metro stop,2019-01-01 06:25:45.000000,0,POINT (52.34528169 4.8587373)


In [18]:
passengerdict = {}

for i in range(len(env.passengers)):
    try:
        passengerdict[env.passengers[i].id] = {"route": env.passengers[i].route, 
                                        "origin": env.passengers[i].route[0], 
                                        "destination": env.passengers[i].route[-1], 
                                        "transfers": 0, 
                                        "waiting_time": 0, 
                                        "modes": 0, 
                                        "transferindex": 0, 
                                        "creation": env.passengers[i].log["Timestamp"][0].strftime ('%H:%M:%S'),
                                        "departure": env.passengers[i].log["Timestamp"][2].strftime ('%H:%M:%S'),
                                        "arrival": env.passengers[i].log["Timestamp"][3].strftime ('%H:%M:%S'), 
                                        "class": 0, 
                                        "date": env.passengers[i].log["Timestamp"][0].date()}
    except:
        pass
    
df = pd.DataFrame(passengerdict).T
df

,arrival,class,creation,date,departure,destination,modes,origin,route,transferindex,transfers,waiting_time
00263f70-6c19-11e9-8d53-7831c1d08f6a,06:31:45,0,05:30:00,2019-01-01,06:06:45,VUMedischCentrum,0,Muntplein,"[Muntplein, Vijzelgracht, DePijp, RoelofHartpl...",0,0,0
00264bd2-6c19-11e9-8cd1-7831c1d08f6a,06:28:30,0,05:30:38,2019-01-01,06:12:45,Rokin,0,Minervaplein,"[Minervaplein, Beethovenstraat, GerritvdVeenst...",0,0,0
00265550-6c19-11e9-91ed-7831c1d08f6a,06:31:45,0,05:30:53,2019-01-01,06:29:45,VUMedischCentrum,0,Amstelveenseweg,"[Amstelveenseweg, VUMedischCentrum]",0,0,0
00269a74-6c19-11e9-b12f-7831c1d08f6a,06:25:45,0,05:32:36,2019-01-01,06:19:00,Olympiaweg,0,GerritvdVeenstraat,"[GerritvdVeenstraat, Beethovenstraat, Minervap...",0,0,0
0026ac6c-6c19-11e9-bc3c-7831c1d08f6a,06:06:45,0,05:33:26,2019-01-01,06:05:45,OlympischStadion,0,IJsbaanpad,"[IJsbaanpad, OlympischStadion]",0,0,0
0026c094-6c19-11e9-9dee-7831c1d08f6a,06:18:45,0,05:33:29,2019-01-01,06:13:30,GerritvdVeenstraat,0,DePijp,"[DePijp, RoelofHartplein, GerritvdVeenstraat]",0,0,0
0026dec6-6c19-11e9-83a2-7831c1d08f6a,06:34:00,0,05:34:17,2019-01-01,06:07:00,CentraalStation,0,OlympischStadion,"[OlympischStadion, Olympiaweg, Olympiaplein, M...",0,0,0
002791c2-6c19-11e9-94ca-7831c1d08f6a,06:30:30,0,05:35:56,2019-01-01,06:22:45,DePijp,0,Minervaplein,"[Minervaplein, Beethovenstraat, GerritvdVeenst...",0,0,0
002811e2-6c19-11e9-866d-7831c1d08f6a,06:06:45,0,05:36:50,2019-01-01,06:00:00,OlympischStadion,0,DeBoelelaanVU,"[DeBoelelaanVU, VUMedischCentrum, Amstelveense...",0,0,0
002847ac-6c19-11e9-8fe0-7831c1d08f6a,06:15:00,0,05:37:55,2019-01-01,06:10:30,GerritvdVeenstraat,0,Olympiaplein,"[Olympiaplein, Minervaplein, Beethovenstraat, ...",0,0,0
